In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
from pytube import YouTube
import pytube
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import math
import cv2
import os
import unicodedata
import string

In [2]:
# Store data in lists
profile_pages = []
production_companies = []
# pages = [str(i) for i in range(1,141000,250)]
pages = ['1']

#Prepare monitoring of loop
start_time = time()
requests = 0
#for every page
for page in pages:

    #make get request
    response = get("https://www.imdb.com/search/title?title_type=feature&languages=en&count=50&start=" + page)

    #pause the loop
    sleep(randint(8,15))

    #monitor requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')
        break

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    del mv_containers[:25]
    # Extract data from indiv. movie containers
    for container in mv_containers:
        if container.find('div', class_ = 'ratings-metascore') is not None:
            profile = container.h3.a['href']
            profile_pages.append(profile)
            #Production Companies
            production_comps = []
            sleep(randint(1,3))
            production_link = profile
            production = get("https://www.imdb.com/" + production_link + "companycredits")
            production_html = BeautifulSoup(production.text, 'html.parser')
            comps_x = production_html.find("div", {"id": "company_credits_content"}).ul.findAll('a')
            for comp in comps_x:
                production_comps.append(comp.text)
            production_companies.append(production_comps)
            

Request:1; Frequency: 0.06949884821547549 requests/s


In [3]:
movie_ratings = pd.DataFrame({
                              'profile': profile_pages,
                              'pro. comp': production_companies
                             })

print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
profile      15 non-null object
pro. comp    15 non-null object
dtypes: object(2)
memory usage: 320.0+ bytes
None


,profile,pro. comp
0,/title/tt9024106/,"[Soli Deo Gloria Releasing, Pure Flix Entertai..."
1,/title/tt1488606/,[Atlas Entertainment]
2,/title/tt8426594/,"[Soapbox Films, Divide/Conquer, Mind Hive Films]"
3,/title/tt4827558/,"[Alcatraz Films, Andrew Lauren Productions (AL..."
4,/title/tt1727824/,"[GK Films, New Regency Pictures, Queen Films L..."
5,/title/tt2386490/,"[DreamWorks Animation, Mad Hatter Entertainment]"
6,/title/tt0167190/,"[Revolution Studios, Lawrence Gordon Productio..."
7,/title/tt4913966/,"[Atomic Monster, New Line Cinema]"
8,/title/tt6966692/,"[Participant Media, DreamWorks, Amblin Partner..."
9,/title/tt8085790/,"[Universal Pictures, Legendary Entertainment, ..."
